In [ ]:
#EDA

import pandas as pd
import sqlite3
conn = sqlite3.connect('vendor_dataset.db')


In [ ]:
tables = pd.read_sql_query("select name from sqlite_master where type = 'table'",conn)
tables

In [ ]:
for table in tables['name']:
    print('-'*50, f'{table}', '-'*50)
    print('Count of records:', pd.read_sql(f"select count(*) as count from {table}",conn)['count'].values[0])
    display(pd.read_sql(f'select * from {table} limit 5',conn))

In [ ]:
purchases = pd.read_sql_query('select * from purchases where VendorNumber = 4466',conn)
purchases

In [ ]:
purchase_prices = pd.read_sql_query('select * from purchase_prices where VendorNumber = 4466',conn)
purchase_prices

In [ ]:
vendor_invoice = pd.read_sql_query('select * from vendor_invoice where VendorNumber = 4466',conn)
vendor_invoice

In [ ]:
sales = pd.read_sql_query('select * from sales where VendorNo = 4466',conn)
sales

In [ ]:
purchases.groupby(['Brand', 'PurchasePrice'])[['Quantity','Dollars']].sum()
purchase_prices

In [ ]:
vendor_invoice['PONumber'].nunique()
vendor_invoice.columns

In [ ]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()

- The purchases table contains actual purchase data, including the date of purchase, products (brands) purchased by vendors, the amount paid (in dollars), and the quantity purchased.
- The purchase price column is derived from the purchase_prices table, which provides product-wise actual and purchase prices. The combination of vendor and brand is unique in this table.
- The vendor_invoice table aggregates data from the purchases table, summarizing quantity and dollar amounts, along with an additional column for freight. This table maintains uniqueness based on vendor and PO number.
- The sales table captures actual sales transactions, detailing the brands purchased by vendors, the quantity sold, the selling price, and the revenue earned.

As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:
- purchase transactions made by vendors
- sales transaction data
- freight costs for each vendor
- actual product prices from vendors

In [ ]:
vendor_invoice.columns

In [ ]:
freight_summary = pd.read_sql_query("""select VendorNumber, SUM(Freight) as FreightCost
from vendor_invoice
group by VendorNumber""",conn)
freight_summary

In [ ]:
vendor_invoice.columns
purchase_prices.columns

In [ ]:
pd.read_sql_query("""
SELECT 
    p.VendorNumber, 
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    pp.Volume,
    pp.Price AS ActualPrice,
    SUM(p.Quantity) AS TotalPurchaseQuantity,
    SUM(p.Dollars) AS TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
    ON p.Brand = pp.Brand 
    where p.PurchasePrice > 0
GROUP BY 
    p.VendorNumber,
    p.VendorName,
    p.Brand
ORDER BY 
    TotalPurchaseDollars;
"""
,conn)

In [ ]:
sales.columns

In [ ]:
pd.read_sql_query("""SELECT 
VendorNo,
Brand,
sum(SalesDollars) as TotalSalesDollars,
sum(SalesPrice) as TotalSalesPrice,
Sum(SalesQuantity) as TotalSalesQuantity,
sum(ExciseTax) as TotalExciseTax
from sales

group by VendorNo,Brand
order by TotalSalesDollars
""",conn)

canot execute this because of big query

In [ ]:
import time
start = time.time()
final_table = pd.read_sql_query("""SELECT
    pp.VendorNumber,
    pp.Brand,
    pp.Price as ActualPrice,
    pp.PurchasePrice,
    SUM(s.SalesQuantity) AS TotalSalesQuantity,
    SUM(s.SalesDollars) AS TotalSalesDollars,
    SUM(s.SalesPrice) AS TotalSalesPrice,
    SUM(s.ExciseTax) AS TotalExciseTax,
    SUM(vi.Quantity) AS TotalPurchaseQuantity,
    SUM(vi.Dollars) AS TotalPurchaseDollars,
    SUM(vi.Freight) AS TotalFreightCost
FROM purchase_prices pp
JOIN sales s
    ON pp.VendorNumber = s.VendorNo
    AND pp.Brand = s.Brand
JOIN vendor_invoice vi
    ON pp.VendorNumber = vi.VendorNumber
GROUP BY pp.VendorNumber, pp.Brand, pp.Price, pp.PurchasePrice
""", conn)


In [ ]:
vendor_sales_summary = pd.read_sql_query("""
WITH FreightSummary AS (
    SELECT 
        VendorNumber,
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),
PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
        AND p.VendorNumber = pp.VendorNumber  -- assuming this joint key is correct
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),
SalesSummary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)

SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost       -- corrected spelling
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC
""", conn)


In [ ]:
vendor_sales_summary


This query generates a vendor-wise sales and purchase summary, which is valuable for:

**Performance Optimization:**

- The query involves heavy joins and aggregations on large datasets like sales and purchases.
- Storing the pre-aggregated results avoids repeated expensive computations.
- Helps in analyzing sales, purchases, and pricing for different vendors and brands.
- Future Benefits of Storing this data for faster Dashboarding & Reporting.
- Instead of running expensive queries each time, dashboards can fetch data quickly from vendor_sales_summary.

In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary['Description'].unique()

In [ ]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64') 

In [ ]:
vendor_sales_summary.fillna(0,inplace = True)

In [ ]:
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [ ]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars'] 

In [ ]:
vendor_sales_summary['ProfitMargin'] =( vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalPurchaseDollars'] ) * 100

In [ ]:
vendor_sales_summary['StockTurnover'] =  vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']

In [ ]:
vendor_sales_summary['SalestoPurchaseRatio'] =  vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
cursor = conn.cursor()

In [ ]:
vendor_sales_summary.columns

In [ ]:
cursor.execute("""create table vendor_sales_summary(
VendorNumber int, 
VendorName varchar(100),
Brand int,
Description varchar(100), 
PurchasePrice decimal(10,2),
ActualPrice decimal(10,2),
Volume,
TotalPurchaseQuantity int,
TotalPurchaseDollars decimal(15,2),
TotalSalesQuantity int,
TotalSalesDollars decimal(15,2),
TotalSalesPrice decimal(15,2),
TotalExciseTax decimal(15,2),
FreightCost decimal(15,2),
GrossProfit decimal(15,2),
ProfitMargin decimal(15,2),
StockTurnover decimal(15,2),
SalestoPurchaseRatio decimal(15,2),
Primary key (VendorNumber, Brand)
);""")

In [ ]:
pd.read_sql_query("select * from vendor_sales_summary",conn)

In [ ]:
vendor_sales_summary.to_sql('vendor_sales_summary',conn, if_exists = 'replace',index = False)

In [ ]:
pd.read_sql_query("select * from vendor_sales_summary",conn)